In [14]:
import tensorflow as tf
from tensorflow.contrib import rnn
import numpy as np
import random

Instructions for updating:
Use the retry module or similar alternatives.


In [15]:
file_name = 'tinyshakespeare.txt'
with open(file_name,'r') as f:
    inputTest = f.read()
    
data_size = len(inputTest)
print(data_size)
individualChars = list(set(inputTest))
vocab_size = len(individualChars)
print(vocab_size)
num_hidden_units = 128

1115394
65


In [17]:
# Convert input into one hot encoded vectors
char_to_ix = {char: idx for idx, char in enumerate(individualChars)}
ix_to_char = {idx : char for idx, char in enumerate(individualChars)}
seq_len = 32
length = 0
data_set = []
word = []
while(length < data_size):
    #print("line : ", inputTest[length])
    charArray = np.zeros((vocab_size,))
    charArray[char_to_ix[inputTest[length]]] = 1
    word.append(charArray)
    length += 1

In [19]:
# Split the data into training sets
batches_train_x = []
batches_train_y = []
length = 0
#while length+seq_len+1<word.shape[0]:
while length+seq_len+1<len(word):
    #print(word[length:(length+seq_len-2)])
    batches_train_x.append(word[length:(length+seq_len)])
    batches_train_y.append(word[length+1:(length+seq_len+1)])
    length = length + seq_len
    #print("length", length)

print(len(batches_train_x))
print(len(batches_train_y))
num_of_batches = len(batches_train_x)
batches_train_x = np.array(batches_train_x)
batches_train_x.reshape([num_of_batches,vocab_size,seq_len])

batches_train_y = np.array(batches_train_y)
batches_train_y.reshape([num_of_batches,vocab_size,seq_len])

34856
34856


array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 1., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

In [21]:
X = tf.placeholder(name='X', dtype=tf.float64, shape= [None, vocab_size, 1])
Y = tf.placeholder(name='Y', dtype=tf.float64, shape= [None, vocab_size])

In [23]:
out_weights=tf.Variable(tf.random_normal([num_hidden_units,vocab_size], dtype=tf.float64), name="out_weights", dtype=tf.float64)
out_bias=tf.Variable(tf.random_normal([vocab_size], dtype=tf.float64), name="out_bias", dtype=tf.float64)

In [25]:
lstm = tf.nn.rnn_cell.LSTMCell(num_hidden_units)
outputs, _ = tf.nn.dynamic_rnn(lstm, X, dtype=tf.float64)
output_inter = tf.transpose(outputs, [1, 0, 2])
last = tf.gather(output_inter, int(output_inter.get_shape()[0]) - 1)

ValueError: Variable rnn/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "<ipython-input-24-3400497fdae1>", line 2, in <module>
    outputs, _ = tf.nn.dynamic_rnn(lstm, X, dtype=tf.float64)
  File "/home/exx/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2910, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/home/exx/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2850, in run_ast_nodes
    if self.run_code(code, result):


In [27]:
prediction = tf.matmul(last, out_weights, name = "softmax") + out_bias
print(prediction)
logits = tf.nn.softmax(prediction)
index = tf.argmax(logits,1)

Tensor("add_1:0", shape=(?, 65), dtype=float64)


In [28]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=prediction,labels=Y))

In [29]:
tf.get_variable_scope().reuse_variables()
optimize = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(loss)

/home/exx/anaconda3/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [30]:
mistakes = tf.not_equal(tf.argmax(Y, 1), tf.argmax(prediction, 1))
error = tf.reduce_mean(tf.cast(mistakes, tf.float32))
correct_prediction=tf.equal(tf.argmax(logits,1),tf.argmax(Y,1))
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    generated_words = []
    for i in range(30):
        for j in range(num_of_batches):
            #print("j : ", j)
            #if j == 0:
                #print("input: ", j, batches_train_x[j])
                #print("label: ", j, batches_train_y[j])
            input_x = np.array(batches_train_x[j])
            input_y = np.array(batches_train_y[j])
            input_x = input_x.reshape(seq_len,vocab_size,1)
            input_y = input_y.reshape(seq_len,vocab_size)
            #print(input_x.shape)
            #print(input_y.shape)
            sess.run(optimize, feed_dict={X:input_x, Y:input_y})
        print("Epoch - ",str(i))
        
        if i%10==0:
            acc=sess.run(accuracy,feed_dict={X:input_x,Y:input_y})
            los=sess.run(loss,feed_dict={X:input_x,Y:input_y})
            print("For iter ",i)
            print("Accuracy ",acc)
            print("Loss ",los)
            print("__________________")
            
        if i%100 == 0:
            sample = np.zeros((vocab_size,1))
            sample[random.randint(0,vocab_size)] = 1
            sample = sample.reshape(1,vocab_size,1)
            
            for k in range(10):
                resulting_logits = sess.run(logits, feed_dict={X:sample})
                resulting_index = sess.run(index, feed_dict={X:sample})
                
                #print(resulting_logits)
                #print(resulting_index[0])
                generated_words.append(ix_to_char[resulting_index[0]]) 
                #print(generated_words)
    print(generated_words)